In [1]:
_list = [1, 2, 3, "a"]
_tuple = (1, 2, 3, "a")
_dict = {}

In [2]:
_list = [1,2,3,4,5]
sum(_list);
sum_val = 0
for x in _list:
    sum_val += x
print(sum_val)

15


In [3]:
[mark1,mark2,mark3] = [1,2,3]
students = {"name1":mark1, "name2":mark2, "name3":mark3}
print(students.keys())
print(students.values())

dict_keys(['name1', 'name2', 'name3'])
dict_values([1, 2, 3])


In [4]:
arr = [[1,2,3],[4,5,6],[7,8,9]]
ans = [g**2 for i in arr for g in i ]
print(ans)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


In [5]:
a = [1,2,3,3,4,3]
a.pop(3)
print(a)


[1, 2, 3, 4, 3]


In [6]:
import sqlite3

In [7]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

Design and implement a Complete University Management System in Python using SQLite and functions (def) with the following requirements:
1. Database (SQLite)
Create the following tables:
Departments (dept_id, dept_name, hod_name)
Students (student_id, name, class, dept_id)
Subjects (subject_id, subject_name, dept_id)
Marks (mark_id, student_id, subject_id, marks)

Insert:
At least 2 departments, 5 students, 4 subjects, and marks for each student in all subjects.
 

In [8]:
cursor.execute("""
    CREATE TABLE Departments(
        dept_id INTEGER,
        dept_name TEXT,
        hod_name TEXT
    )
""")

cursor.execute("""

    CREATE TABLE Students(
        student_id INTEGER,
        name TEXT,
        class INTEGER,
        dept_id INTEGER
    )
""")

cursor.execute("""

        CREATE TABLE Subjects(
        subject_id INTEGER,
        subject_name TEXT,
        dept_id INTEGER
    )
""")

cursor.execute("""

        CREATE TABLE Marks(
        mark_id INTEGER,
        student_id INTEGER,
        subject_id INTEGER,
        marks
    )
""")

In [9]:
departments = ["Engineering","Arts"]
hod = ["misterA", "misterB"]

students = ["student_1", "student_2", "student_3", "student_4", "student_5"]
subjects = ["Math", "Science", "English", "History"]
marks = range(40, 100, 3)

dept_data = list(zip(range(2),departments,hod))
student_data = list(zip(range(5), students, [1,1,2,2,3],[0,1,0,1,0]))
subject_data = list(zip(range(4), subjects, [0,0,1,1]))
mark_data = list(zip(range(20), list(range(5)) * 4, list(range(4)) * 5, marks))
print(mark_data)

[(0, 0, 0, 40), (1, 1, 1, 43), (2, 2, 2, 46), (3, 3, 3, 49), (4, 4, 0, 52), (5, 0, 1, 55), (6, 1, 2, 58), (7, 2, 3, 61), (8, 3, 0, 64), (9, 4, 1, 67), (10, 0, 2, 70), (11, 1, 3, 73), (12, 2, 0, 76), (13, 3, 1, 79), (14, 4, 2, 82), (15, 0, 3, 85), (16, 1, 0, 88), (17, 2, 1, 91), (18, 3, 2, 94), (19, 4, 3, 97)]


In [10]:
cursor.executemany("INSERT INTO Departments(dept_id, dept_name, hod_name) VALUES (?, ? , ?)", dept_data)
cursor.executemany("INSERT INTO Students(student_id, name, class, dept_id) VALUES (?, ? , ?, ?)", student_data)
cursor.executemany("INSERT INTO Subjects(subject_id, subject_name, dept_id) VALUES (?, ? , ?)", subject_data)
cursor.executemany("INSERT INTO Marks(mark_id, student_id, subject_id, marks) VALUES (?, ? , ?, ?)", mark_data)


2. Python Functions
Implement the following:
def add_student(name, class, dept_id)
def add_subject(subject_name, dept_id)
def add_marks(student_id, subject_id, marks)
def show_students() → Display students with department name (JOIN)
def show_subjects() → Display all subjects with their department
def show_marks() → Display each student’s marks with subject names (JOIN)
def student_report(student_id) → Show subject-wise marks, total, average, grade, pass/fail
def toppers() → Show topper in each subject + overall topper
def department_summary() → Show average marks department-wise (JOIN + GROUP BY)
def unique_subjects() → Return unique subjects as a set

In [91]:
def add_student(name, class_name, dept_id):
    cursor.execute("INSERT INTO Students(name, class, dept_id) VALUES (? , ?, ?)", (name, class_name, dept_id))
    
def add_subject(subject_name, dept_id):
    cursor.execute("INSERT INTO Subjects(subject_name, dept_id) VALUES (? , ?)", (subject_name, dept_id))

def add_marks(student_id, subject_id, marks):
    cursor.execute("INSERT INTO Marks(student_id, subject_id, marks) VALUES (?, ? , ?, ?)", (student_id, subject_id, marks))


def show_students():
    return cursor.execute("""
        SELECT st.student_id, st.name, st.class, d.dept_name
        FROM Students st
        INNER JOIN Departments d ON st.dept_id = d.dept_id
    """).fetchall()

def show_subjects():
    return cursor.execute("""
        SELECT sb.subject_id, sb.subject_name, d.dept_name
        FROM Subjects sb
        INNER JOIN Departments d ON sb.dept_id = d.dept_id
    """).fetchall()
def show_marks():
    return cursor.execute("""
        SELECT m.student_id, sb.subject_name, m.marks
        FROM Marks m
        INNER JOIN Subjects sb ON m.subject_id = sb.subject_id
    """).fetchall()


def student_report(student_id):
    #Show subject-wise marks, total, average, grade, pass/fail
    letters = {'6':'D', '7':'C', '8':'B', '9':'A'}
    temp = cursor.execute("""
        SELECT sb.subject_name, m.marks
        FROM Marks m
        INNER JOIN Subjects sb ON m.subject_id = sb.subject_id
         where m.student_id = ?
    """, (student_id)).fetchall()
    names, grades = zip(*temp)
    letter_grades = []
    pass_fail = []
    for grade in grades:
        val = int(grade/10)        
        if val >= 6:
            letter_grades.append(letters[str(val)])
        else:
            letter_grades.append('F')
        passing = True if val >= 7 else False
        pass_fail.append(passing)
    total = sum(list(grades))
    avg = total/len(list(grades))
    temp = list(temp)
    return [*zip(names,grades,letter_grades,pass_fail), ('Total', total), ('Average', avg)]

def toppers():
    #→ Show topper in each subject + overall topper    
    result = []
    for subject_id in range(len(subjects)):
        result.append(cursor.execute("""
        SELECT s.name, sb.subject_name, MAX(m.marks)
        FROM Marks m INNER JOIN Students s ON m.student_id = s.student_id
        INNER JOIN Subjects sb on m.subject_id = sb.subject_id
        where m.subject_id = ?
    """, (str(subject_id))).fetchall())
    return result
    
def department_summary():
    #→ Show average marks department-wise (JOIN + GROUP BY) 
    return cursor.execute("""
        SELECT d.dept_name, AVG(m.marks)
        FROM Departments d INNER JOIN Subjects sb on d.dept_id = sb.dept_id
        INNER JOIN Marks m ON sb.subject_id = m.subject_id  
        GROUP BY d.dept_id
    """).fetchall()
    
def unique_subjects():
    #→ Return unique subjects as a set
    temp = cursor.execute("""
        SELECT sb.subject_name
        FROM Subjects sb
    """).fetchall()
    return set(*zip(*temp))

In [12]:
show_students()

[(0, 'student_1', 1, 'Engineering'),
 (1, 'student_2', 1, 'Arts'),
 (2, 'student_3', 2, 'Engineering'),
 (3, 'student_4', 2, 'Arts'),
 (4, 'student_5', 3, 'Engineering')]

In [13]:
show_subjects()

[(0, 'Math', 'Engineering'),
 (1, 'Science', 'Engineering'),
 (2, 'English', 'Arts'),
 (3, 'History', 'Arts')]

In [14]:
show_marks()

[(0, 'Math', 40),
 (1, 'Science', 43),
 (2, 'English', 46),
 (3, 'History', 49),
 (4, 'Math', 52),
 (0, 'Science', 55),
 (1, 'English', 58),
 (2, 'History', 61),
 (3, 'Math', 64),
 (4, 'Science', 67),
 (0, 'English', 70),
 (1, 'History', 73),
 (2, 'Math', 76),
 (3, 'Science', 79),
 (4, 'English', 82),
 (0, 'History', 85),
 (1, 'Math', 88),
 (2, 'Science', 91),
 (3, 'English', 94),
 (4, 'History', 97)]

In [62]:
student_report('1')

43
4.3
58
5.8
73
7.3
88
8.8


[('Science', 43, 'F', False),
 ('English', 58, 'F', False),
 ('History', 73, 'C', True),
 ('Math', 88, 'B', True),
 ('Total', 262),
 ('Average', 65.5)]

In [75]:
toppers()

[[('student_2', 'Math', 88)],
 [('student_3', 'Science', 91)],
 [('student_4', 'English', 94)],
 [('student_5', 'History', 97)]]

In [83]:
department_summary()

[('Engineering', 65.5), ('Arts', 71.5)]

In [92]:
unique_subjects()

{'English', 'History', 'Math', 'Science'}